In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By     # 태그 가져올 때
from selenium.webdriver import Keys             # pageDown 키를 누를 때 필요

In [9]:
# 로딩하는 시간 
import time

In [10]:
## 드라이버 설정
chrom_options = webdriver.ChromeOptions()               # 여기 옵션을 추가하여 브라우저 설정을 할 수 있습니다.
chrom_options.add_experimental_option('detach', True)   # 브라우저 꺼지는 것을 방지
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                          options = chrom_options)

In [11]:
url = "https://store.steampowered.com/charts/topselling/global"
driver.get(url)
time.sleep(5)


In [12]:
import pandas as pd
import csv

In [13]:
title_elements = driver.find_elements(By.CSS_SELECTOR, "._1n_4-zvf0n4aqGEksbgW9N")
price_elements = driver.find_elements(By.CSS_SELECTOR, ".Wh0L8EnwsPV_8VAu8TOYr")

result_title = []
result_price = []
result_weeks = []

for t in title_elements:
    result_title.append(t.text)

for t in price_elements:
    result_price.append(t.text)


# 결과 딕셔너리 초기화

df = pd.DataFrame({
    '게임 이름' : result_title,
    '게임 가격' : result_price
})

# CSV 파일로 저장
csv_file = 'steam_games.csv'
df.to_csv(csv_file, index=False, encoding='utf-8-sig')

